In [3]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Load data
data = pd.read_csv('../dataset/preprocessed_data.csv')

In [6]:
# Define numerical columns
num_cols = ['minutes', 'n_steps', 'n_ingredients', 'calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']

# Create user-item matrix
n_users = data['user_id'].nunique()
n_items = data['recipe_id'].nunique()
user_item_matrix = np.zeros((n_users, n_items))
for row in data.itertuples():    
    user_item_matrix[int(row[1]-1), int(row[14]-1)] = row[15]

In [7]:
# Perform matrix factorization using NMF
k = 10  # Number of latent features to extract
model = NMF(n_components=k, max_iter=100)
W = model.fit_transform(user_item_matrix)
H = model.components_

: 

: 

In [ ]:
# Compute item-item similarity using cosine similarity
item_item_similarity = cosine_similarity(H.T)

In [ ]:
# Define function to get similar recipe_ids based on user-item interactions and other features
def recommend_similar_recipes(user_id, recipe_id, top_n=5):
    user_idx = user_id - 1
    recipe_idx = recipe_id - 1
    
    # Get latent features for user
    user_latent_features = W[user_idx, :]
    
    # Get latent features for recipe
    recipe_latent_features = H[:, recipe_idx]
    
    # Combine latent features with other features
    recipe_features = np.concatenate((recipe_latent_features, data.loc[recipe_idx, num_cols].values))
    
    # Compute similarity between recipe and other recipes
    similarities = cosine_similarity([recipe_features], np.concatenate((H.T, data[num_cols]), axis=1))[0]
    
    # Get top n most similar recipe_ids
    similar_recipe_ids = np.argsort(-similarities)[:top_n]
    
    return similar_recipe_ids + 1  # Add 1 to convert back to 1-indexing


In [ ]:

# Example usage
user_id = 492
recipe_id = 20636
similar_recipe_ids = recommend_similar_recipes(user_id, recipe_id, top_n=5)
print(similar_recipe_ids)